In [ ]:
import torch
import torch.nn as nn
import torch.optim as op
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as f
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#Extracting the dataset zip file
!unrar e '/content/drive/MyDrive/IvLabs/CNN Dataset/Doodle dataset.rar'

In [ ]:
#Shifting to GPUs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
d1=np.load("/content/full_numpy_bitmap_airplane.npy")
l1=np.full((d1.shape[0],1),0)
d2=np.load("/content/full_numpy_bitmap_ant.npy")
l2=np.full((d2.shape[0],1),1)
d3=np.load("/content/full_numpy_bitmap_banana.npy")
l3=np.full((d3.shape[0],1),2)
d4=np.load("/content/full_numpy_bitmap_baseball.npy")
l4=np.full((d4.shape[0],1),3)
d5=np.load("/content/full_numpy_bitmap_bird.npy")
l5=np.full((d5.shape[0],1),4)
d6=np.load("/content/full_numpy_bitmap_bucket.npy")
l6=np.full((d6.shape[0],1),5)
d7=np.load("/content/full_numpy_bitmap_butterfly.npy")
l7=np.full((d7.shape[0],1),6)
d8=np.load("/content/full_numpy_bitmap_cat.npy")
l8=np.full((d8.shape[0],1),7)
d9=np.load("/content/full_numpy_bitmap_coffee cup.npy")
l9=np.full((d9.shape[0],1),8)
d10=np.load("/content/full_numpy_bitmap_dolphin.npy")
l10=np.full((d10.shape[0],1),9)
d11=np.load("/content/full_numpy_bitmap_donut.npy")
l11=np.full((d11.shape[0],1),10)
d12=np.load("/content/full_numpy_bitmap_duck.npy")
l12=np.full((d12.shape[0],1),11)
d13=np.load("/content/full_numpy_bitmap_fish.npy")
l13=np.full((d13.shape[0],1),12)
d14=np.load("/content/full_numpy_bitmap_leaf.npy")
l14=np.full((d14.shape[0],1),13)
d15=np.load("/content/full_numpy_bitmap_mountain.npy")
l15=np.full((d15.shape[0],1),14)
d16=np.load("/content/full_numpy_bitmap_pencil.npy")
l16=np.full((d16.shape[0],1),15)
d17=np.load("/content/full_numpy_bitmap_smiley face.npy")
l17=np.full((d17.shape[0],1),16)
d18=np.load("/content/full_numpy_bitmap_snake.npy")
l18=np.full((d18.shape[0],1),17)
d19=np.load("/content/full_numpy_bitmap_umbrella.npy")
l19=np.full((d19.shape[0],1),18)
d20=np.load("/content/full_numpy_bitmap_wine bottle.npy")
l20=np.full((d20.shape[0],1),19)
dataset_whole=np.concatenate((d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20),axis=0)
label_whole=np.concatenate( (l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13,l14,l15,l16,l17,l18,l19,l20),axis=0)


In [ ]:
#Custom Dataset
class Data(Dataset):
  def __init__(self):
    self.x=dataset_whole.reshape(dataset_whole.shape[0],1,28,28)
    
    self.y=label_whole.reshape(dataset_whole.shape[0],)
    self.n=dataset_whole.shape[0]
  def __getitem__(self,i):
    return self.x[i],self.y[i]
  def __len__(self):
    return self.n
data=Data()
train_data, test_data = torch.utils.data.random_split(data, [1650000,dataset_whole.shape[0]-1650000 ])
dataloader=DataLoader(train_data,batch_size=10000,shuffle=True)

dataloader_test=DataLoader(test_data,batch_size=10000,shuffle=True)


In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__() 
    self.c1=nn.Conv2d(1,8,5,padding=1)
    self.mp1=nn.MaxPool2d(2,2)
    self.c2=nn.Conv2d(8,16,5,padding=1)
    self.c3=nn.Conv2d(16 ,28, 3, padding=1)
    self.c4=nn.Conv2d(28,48,3,padding=1)
    self.l1=nn.Linear(48*5*5,500)
    self.do=nn.Dropout(0.25)
    self.l2=nn.Linear(500,250)
    self.l3=nn.Linear(250,20)
    
  def forward(self,x):
    x= self.mp1(f.relu(self.c1(x)))
    x= self.mp1(f.relu(self.c2(x)))
    x= f.relu(self.c3(x))
    x= f.relu(self.c4(x))
    x=x.view(-1,48*5*5)
    x= (f.relu(self.l1(x)))
    x= (f.relu(self.l2(x)))
    x= self.l3(x)  
    return x
    
Doodle=Net().to(device)
loss =nn.CrossEntropyLoss()
opt= op.SGD(Doodle.parameters(),lr=0.01)

In [ ]:
cost_values=[]
for j in range(100):
  for i,(data , target) in enumerate(dataloader):
    data=torch.tensor(data).to(device)
    target=torch.tensor(target).to(device)
    opt.zero_grad()
    out=Doodle(data.float())
    cost=loss(out,target)
    cost.backward()
    opt.step()
    
  print("Epoch--> ",j," Cost--> ",cost.item())
  cost_values.append(cost.item())
plt.plot(cost_values)
plt.xlabel("number of Epochs")
plt.ylabel("Loss Value")


In [ ]:
#Saving the model 
File="doodle.pth"
torch.save(Doodle.state_dict(),File)

In [ ]:
#Accuracy
c=0
for i,(data , target) in enumerate(dataloader_test):
 
  target=target.to(device)
  out=Doodle(torch.tensor(data).to(device).float())
  n=torch.argmax(out,axis=1).to(device)
  r=n==target
  a=torch.ones(target.shape).to(device)
  r=a*r
  c+=torch.sum(r).item()
print("Test accuracy--> ",(c/(dataset_whole.shape[0]-1650000 ))*100)

c=0
for j,(data , target) in enumerate(dataloader):
  target=target.to(device)
  out=Doodle(torch.tensor(data).to(device).float())
  n=torch.argmax(out,axis=1).to(device)
  r=n==target
  a=torch.ones(target.shape).to(device)
  r=a*r
  c+=torch.sum(r).item()
print("train accuracy--> ",(c/1650000)*100)
